In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Practice').getOrCreate()

24/04/24 01:47:24 WARN Utils: Your hostname, elena-VB resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/04/24 01:47:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/24 01:47:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [5]:
df = spark.read.csv('class_grades.csv', header=True, inferSchema=True)

In [6]:
df.show(10)

+------+----------+--------+-------+--------+-----+
|Prefix|Assignment|Tutorial|Midterm|TakeHome|Final|
+------+----------+--------+-------+--------+-----+
|     5|     57.14|   34.09|  64.38|   51.48| 52.5|
|     8|     95.05|  105.49|   67.5|   99.07|68.33|
|     8|      83.7|   83.17|   30.0|   63.15|48.89|
|     7|     81.22|   96.06|  49.38|  105.93|80.56|
|     8|     91.32|   93.64|   95.0|  107.41|73.89|
|     7|      95.0|   92.58|  93.12|   97.78|68.06|
|     8|     95.05|  102.99|  56.25|   99.07| 50.0|
|     7|     72.85|   86.85|   60.0|    NULL|56.11|
|     8|     84.26|    93.1|   47.5|   18.52|50.83|
|     7|      90.1|   97.55|  51.25|   88.89|63.61|
+------+----------+--------+-------+--------+-----+
only showing top 10 rows



In [7]:
df = df.na.drop()

In [33]:
df = df.drop('Assignment', 'Tutorial', 'Takehome')

In [34]:
df.show(10)

+------+-------+-----+
|Prefix|Midterm|Final|
+------+-------+-----+
|     5|  64.38| 52.5|
|     8|   67.5|68.33|
|     8|   30.0|48.89|
|     7|  49.38|80.56|
|     8|   95.0|73.89|
|     7|  93.12|68.06|
|     8|  56.25| 50.0|
|     8|   47.5|50.83|
|     7|  51.25|63.61|
|     7|   75.0|39.72|
+------+-------+-----+
only showing top 10 rows



In [35]:
df.printSchema()

root
 |-- Prefix: integer (nullable = true)
 |-- Midterm: double (nullable = true)
 |-- Final: double (nullable = true)



In [36]:
from pyspark.ml.feature import VectorAssembler

In [37]:
assembler = VectorAssembler().setInputCols(['Midterm']).setOutputCol('MidtermVector')

In [38]:
data = assembler.transform(df)

In [39]:
from pyspark.ml.regression import LinearRegression

In [40]:
train_data,test_data = data.randomSplit([0.75,0.25])

In [41]:
regressor = LinearRegression(featuresCol='MidtermVector', labelCol='Final')

In [42]:
regressor = regressor.fit(train_data)

24/04/24 02:07:31 WARN Instrumentation: [fb27737a] regParam is zero, which might cause numerical instability and overfitting.


In [43]:
regressor.coefficients

DenseVector([0.6238])

In [44]:
regressor.intercept

25.973306503019646

In [45]:
pred = regressor.evaluate(test_data)

In [46]:
pred.predictions.show()

+------+-------+------+-------------+------------------+
|Prefix|Midterm| Final|MidtermVector|        prediction|
+------+-------+------+-------------+------------------+
|     4|  38.75| 49.17|      [38.75]| 50.14576057557167|
|     6|   72.5| 68.33|       [72.5]|  71.1991883161815|
|     7|  41.25| 39.72|      [41.25]|51.705273741542776|
|     7|  41.25| 46.67|      [41.25]|51.705273741542776|
|     7|   75.0| 39.72|       [75.0]|  72.7587014821526|
|     7|  83.12| 75.83|      [83.12]| 77.82400024522674|
|     7|  86.25| 83.33|      [86.25]| 79.77651072902255|
|     7|  96.25| 88.89|      [96.25]| 86.01456339290695|
|     7|  99.38|  95.0|      [99.38]| 87.96707387670276|
|     7| 101.25|107.78|     [101.25]| 89.13358972484914|
|     7| 106.25|102.78|     [106.25]| 92.25261605679134|
|     8|   37.5| 28.06|       [37.5]| 49.36600399258613|
|     8|   47.5| 50.83|       [47.5]| 55.60405665647052|
|     8|  56.25|  50.0|      [56.25]|61.062352737369366|
|     8|  56.25| 66.39|      [5

In [48]:
pred.meanAbsoluteError

9.00021581863453

In [49]:
pred.meanSquaredError

143.2197769348384